In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

import torch
import torch.nn as nn
from torch.functional import F
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.data import Data, HeteroData
from torch_geometric.loader import DataListLoader, DataLoader
from torch_geometric.utils.convert import to_networkx
import networkx as nx

from sklearn.preprocessing import MinMaxScaler

/var/folders/0t/9z26630n32l3qw5w80zvvjqw0000gn/T/ipykernel_10880/1010190927.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Load network data: node features, edge_features and edge_index
node_features = pd.read_csv('network/node_features.csv', index_col=0, header=0).astype(float).fillna(value=0)
branch_index = pd.read_csv('network/branch_index.csv', index_col=0, header=0)
trafo_index = pd.read_csv('network/trafo_index.csv', index_col=0, header=0)
branch_attr = pd.read_csv('network/branch_attr.csv', index_col=0, header=0)
trafo_attr = pd.read_csv('network/trafo_attr.csv', index_col=0, header=0)

In [3]:
node_features.shape, branch_index.shape, trafo_index.shape, branch_attr.shape, trafo_attr.shape

((118, 8), (173, 2), (13, 2), (173, 3), (13, 5))

In [4]:
branch_index = torch.tensor(branch_index.to_numpy().T, dtype=torch.long)
trafo_index = torch.tensor(trafo_index.to_numpy().T, dtype=torch.long)

In [6]:


# Load synthetic 'p_mw' at load buses
p_load_data = pd.read_csv('synthetic_data/p_load_data.csv', index_col=0, header=0)
p_load_data.shape

NameError: name 're_gen_no' is not defined

In [ ]:
# Load synthetic 'p_mw' at generator buses
p_gen_data = pd.read_csv('synthetic_data/p_gen_data.csv', index_col=0, header=0)
p_gen_data.shape

(53, 1000)

In [ ]:
# Load scaled 'p_mw' at all buses
p_bus_data = pd.read_csv('synthetic_data/p_bus_data.csv', index_col=0, header=0)
p_bus_data.shape

(118, 1000)

In [ ]:
p_slack_max_data = pd.read_csv('synthetic_data/p_slack_max_data.csv', index_col=0, header=0)
p_slack_max_data.shape

(1000, 1)

In [ ]:
# Load the list of renewable generator buses
re_gen_bus = pd.read_csv('synthetic_data/re_gen_bus.csv', index_col=0, header=0).to_numpy().reshape((-1))
re_gen_no = pd.read_csv('synthetic_data/re_gen_no.csv', index_col=0, header=0).to_numpy().astype(bool).reshape((-1))
not_re_gen_no = ~re_gen_no

# Load gen bus no
gen_no = pd.read_csv('synthetic_data/gen_no.csv', index_col=0, header=0).to_numpy().astype(bool).reshape((-1))
load_no = pd.read_csv('synthetic_data/load_no.csv', index_col=0, header=0).to_numpy().astype(bool).reshape((-1))
# load_gen_bus = pd.read_csv('synthetic_data/load_gen_bus.csv', index_col=0, header=0).to_numpy().astype(bool).reshape((-1))

In [ ]:
re_gen_bus.shape, re_gen_no.shape, not_re_gen_no.shape, gen_no.shape, load_no.shape

((16,), (53,), (53,), (118,), (118,))

#### Node prediction dataset

In [ ]:
# Store multiple Data() in a list
data_list = []

for i in range(p_load_data.shape[1]):
    node_features['load_p_mw'][load_no] = p_load_data.iloc[:, i].values
    node_features['max_gen_p_mw'][re_gen_bus] = p_gen_data.iloc[re_gen_no, i].values
    node_features['min_gen_p_mw'][re_gen_bus] = p_gen_data.iloc[re_gen_no, i].values
    # slack bus no = 68
    node_features['max_gen_p_mw'][68] = p_slack_max_data.iloc[i].values

    X = node_features.to_numpy()
    y = p_bus_data.iloc[:, i].to_numpy()
    X = torch.from_numpy(X).float()
    y = torch.from_numpy(y.reshape((-1))).float()

    data = HeteroData()
    data['node'].x = X
    data['node'].y = y

    # Set edge index
    data['node', 'branch', 'node'].edge_index = branch_index
    data['node', 'trafo', 'node'].edge_index = trafo_index
    data['node', 'branch', 'node'].edge_attr = torch.from_numpy(branch_attr.to_numpy()).float()
    data['node', 'trafo', 'node'].edge_attr = torch.from_numpy(trafo_attr.to_numpy()).float()
    
    data_list.append(data)

In [ ]:
## Save train and test dataset
train_size = int(0.7*len(data_list))
val_size = int(0.1*len(data_list))
test_size = int(0.2*len(data_list))

# Create PyTorch Geometric DataLoader()
batch_size = 1
train_data_loader = DataLoader(dataset=data_list[:train_size], batch_size=batch_size, shuffle=False)
val_data_loader = DataLoader(dataset=data_list[train_size:train_size+val_size], batch_size=batch_size, shuffle=False)
test_data_loader = DataLoader(dataset=data_list[-test_size:], batch_size=batch_size, shuffle=False)

# Save train and test datasets
torch.save(train_data_loader, 'train_test_dataset/node_prediction_train_dataset.pt')
torch.save(val_data_loader, 'train_test_dataset/node_prediction_val_dataset.pt')
torch.save(test_data_loader, 'train_test_dataset/node_prediction_test_dataset.pt')